In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
from src.Model import Model


In [9]:
from src.Feedback import Feedback
from src.Block import Block

In [10]:
model = Model(verbose=False)

ggml_backend_cuda_buffer_type_alloc_buffer: allocating 25145.55 MiB on device 0: cudaMalloc failed: out of memory
llama_model_load: error loading model: failed to allocate buffer
llama_load_model_from_file: failed to load model


ValueError: Failed to load model from file: /home/suraj/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf

In [ ]:
block_dir_path = "/home/suraj/MedCoQ/newversion/data/blocks/clarity"
block = Block("ClarityBlock", model, block_dir_path)

In [ ]:
question = block.questions[0]
feedback = Feedback(question)
print(feedback.make_query("", "some text"))

Role: You are a medical expert at understanding and writing in the English language.
Goal: Your goal is to check whether the given text is assertive or not. Always try to first identify problems in the text before generating a response and a feedback. If the text is assertive then respond with “No” with a feedback. If the text is not assertive then respond with “Yes” with no other text.

Definitions:
- Assertive Text: An assertive text does not instill doubt in the reader and can be confidently presented in sensitive situations. The language of the assertive text is easy to follow and understand.

Follow the below instructions carefully:
- Always generate a "Response"
- Strictly follow the format of the examples.
- A text can contain phrases like “may or may not be” or “maybe”, in this case always use your best judgment to infer if the information associated with these phrases can be presented in a more assertive language.

Follow the below instructions to generate feedback:
- Feedback

In [ ]:
output = model.respond(feedback.make_query("", "Giant Axonal Neuropathy (GAN) is a rare inherited neurological disorder that affects the peripheral and central nervous systems. The gene that is known to be involved in this condition is called the gigaxonin gene, which is located on chromosome 16 (16q24.1)."))
print(output)

There are no problematic parts in the text. It clearly and confidently describes Giant Axonal Neuropathy (GAN) as a rare inherited neurological disorder, specifies the gene and chromosome associated with it, and there's no ambiguous language used.

Response: Yes
Feedback: None


The text states that the character of the pulse is "not influenced" by several factors, but then goes on to define one of those factors (rate), which suggests that it does in fact influence the character of the pulse. This creates confusion and undermines the assertiveness of the statement.

Response: No
Feedback: Make the text more assertive by clarifying that some factors, including rate, can influence the character of the pulse.


In [11]:
from src.utils import extract_problems, extract_feedback, extract_response

In [12]:
import re
def extract_response(text: str):
    regex = r".*Response:"
    after_regex = r".*\n*"

    if "Response:" not in text: return "yes"    
    
    text = re.sub(regex, "", text, flags=re.DOTALL).strip().lower()
    matches = re.search(after_regex, text, re.MULTILINE)
    
    if matches:
        result = matches.group().strip()
        if "no" in result: return "no"
        
    return "yes"

extract_response(output)

'yes'

In [13]:
extract_problems(output)

"There are no problematic parts in the text. It clearly and confidently describes Giant Axonal Neuropathy (GAN) as a rare inherited neurological disorder, specifies the gene and chromosome associated with it, and there's no ambiguous language used."

In [14]:
extract_feedback(output)

'None'